# libraries

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

import torchmetrics
import torchvision
from torchvision import transforms as T
from torchvision import datasets
from torch.utils.data import DataLoader

import tqdm
import matplotlib.pyplot as plt
import torch.optim as optim

# Dataset

In [2]:
transform = T.Compose([
    T.ToTensor(),
    T.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_folder = './train'
test_folder ='./test'

train_dataset = datasets.ImageFolder(train_folder, transform=transform)
test_dataset = datasets.ImageFolder(test_folder, transform=transform)

# Crea DataLoader per 'train' e 'test'
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


# CNN

In [3]:
class CustomCNN_1(nn.Module):
    def __init__(self, num_classes=5):
        super().__init__()

        # Convolutional layers
        self.conv1 = nn.Conv2d(3, 16, kernel_size=5, padding=2)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2)
        self.bn1 = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=5, padding=2)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2)
        self.bn2 = nn.BatchNorm2d(32)

        # Fully connected layers
        self.fc1 = nn.Linear(32 * 24 * 24, 100)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(100, num_classes)
        self.drop= nn.Dropout(0.3)

    def forward(self, x):
        # Convolutional layers
        x = self.pool1(self.relu1(self.bn1((self.conv1(x)))))
        x = self.pool2(self.relu2((self.bn2(self.conv2(x)))))

        # Flatten for fully connected layers
        x = x.reshape(-1, 32 * 24 * 24)

        # Fully connected layers
        x = self.drop(self.relu3(self.fc1(x)))
        x = self.fc2(x)

        return x

# Initialize your model
model = CustomCNN_1()

# Define your loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

# Train

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
#print(device)
cnn = CustomCNN_1().to(device)

In [5]:
def accuracy(net, Loader, device):
  acc = torchmetrics.Accuracy(task = "multiclass", num_classes= 5).to(device)
  for xb, yb in train_loader:
      xb, yb = xb.to(device), yb.to(device)
      ypred = net(xb)
      _ = acc(ypred, yb)
  return acc.compute()

In [6]:
#num_classes=5
accuracy(cnn, train_loader, device)
loss = nn.CrossEntropyLoss()
opt = torch.optim.Adam(cnn.parameters())

In [7]:
for epoch in range(3):
  
  cnn.train()
  for xb, yb in tqdm.tqdm(train_loader):
    
    xb, yb = xb.to(device), yb.to(device)

    optimizer.zero_grad()
    ypred = cnn(xb)
    l = loss(ypred, yb)
    l.backward()
    optimizer.step()

  cnn.eval()
  print(f'Accuracy at epoch {epoch}: {accuracy(cnn, test_loader, device)}')

100%|██████████| 200/200 [00:36<00:00,  5.46it/s]


Accuracy at epoch 0: 0.3002041280269623


100%|██████████| 200/200 [00:36<00:00,  5.51it/s]


Accuracy at epoch 1: 0.29910504817962646


100%|██████████| 200/200 [00:38<00:00,  5.23it/s]


Accuracy at epoch 2: 0.299576073884964
